In [1]:
# SARSA和Q-learning算法
import time
from yuanyangEnv import YuanYangEnv
import numpy as np

pygame 2.1.2 (SDL 2.0.18, Python 3.9.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class TD_RL:
    def __init__(self,yuanyang):
        self.yuanyang = yuanyang
        self.qvalue = np.zeros((len(self.yuanyang.states), len(self.yuanyang.actions)))
        self.gamma = yuanyang.gamma
        self.actions = yuanyang.actions

    def greedy_policy(self,qfun,state):
        # 贪婪策略
        amax = qfun[state,:].argmax()
        return self.actions[amax]

    def epsilon_greedy_policy(self,qfun,state,epsilon):
        # ε-greedy策略
        amax = qfun[state,:].argmax()
        if np.random.uniform() < 1 - epsilon:
            return self.yuanyang.actions[amax]
        else:
            return self.actions[int(np.random.rand()*len(self.actions))]

    def find_anum(self,a):
        # 找到动作对应的序号
        for i in range(len(self.actions)):
            if a == self.actions[i]:
                return i

    def greedy_test(self):
        # 当测试初始状态为0时，采用当前的贪婪策略能否找到目标点
        s = 0
        s_sample = []
        done = False
        flag = 0
        step_num = 0
        while False == done and step_num < 30:
            a = self.greedy_policy(self.qvalue,s)
            # 与环境交互
            s_next, r, done = self.yuanyang.transform(s, a)
            s_sample.append(s)
            s = s_next
            step_num += 1
        if s == 9:
            flag = 1
        if s == 9 and step_num < 21:
            # 找到最短路径
            flag = 2
        return flag

    def sarsa(self,num_iter,alpha,epsilon):
        # 同策略时间差分强化学习算法SARSA
        iter_num = []
        self.qvalue = np.zeros((len(yuanyang.states),len(yuanyang.actions)))
        
        # 外循环
        for iter in range(num_iter):
            epsilon = epsilon * 0.99
            s_sample = []
            s = 0
            flag = self.greedy_test()
            if flag == 1:
                iter_num.append(iter)
                if len(iter_num) < 2:   # 第一次找到终点
                    print('sarsa第一次完成任务需要的迭代次数为:',iter_num[0])
            if flag == 2:
                print('sarsa第一次实现最短路径需要的迭代次数为:', iter)
                break
                
            # epsilon_greedy策略选择动作
            a = self.epsilon_greedy_policy(self.qvalue, s, epsilon)
            done = False
            count = 0
            # 第二个循环，一个实验，s0-s1-s2-s1-s2-s_terminate
            while False == done and count < 30:
                # 与环境交互
                s_next, r, done = self.yuanyang.transform(s, a)
                a_num = self.find_anum(a)
                if s_next in s_sample:
                    r = -2
                s_sample.append(s)
                
                # 判断一下,是否是终止状态
                if done == True:
                    q_target = r
                else:
                    # 下一个状态处的最大动作,这个地方体现on-policy
                    a1 = self.epsilon_greedy_policy(self.qvalue, s_next, epsilon)
                    a1_num = self.find_anum(a1)
                    # 更新公式
                    q_target = r + self.gamma * self.qvalue[s_next, a1_num]
                    
                # 利用td方法更新动作值函数，alpha
                self.qvalue[s, a_num] = self.qvalue[s, a_num] + alpha * (q_target - self.qvalue[s, a_num])
                
                s = s_next
                a = self.epsilon_greedy_policy(self.qvalue, s, epsilon)
                count += 1
        return self.qvalue

    def qlearning(self, num_iter, alpha, epsilon):
        iter_num = []
        self.qvalue = np.zeros((len(self.yuanyang.states), len(self.yuanyang.actions)))
        
        # 外循环
        for iter in range(num_iter):
            s = 0
            flag = self.greedy_test()
            if flag == 1:
                iter_num.append(iter)
                if len(iter_num) < 2:
                    print("qlearning 第一次完成任务需要的迭代次数为：", iter_num[0])
            if flag == 2:
                print("qlearning 第一次实现最短路径需要的迭代次数为：", iter)
                break
                
            s_sample = []
            a = self.epsilon_greedy_policy(self.qvalue, s, epsilon)
            done = False
            count = 0
            while False == done and count < 30:
                # 与环境交互得到下一个状态
                s_next, r, done = yuanyang.transform(s, a)
                a_num = self.find_anum(a)
                
                if s_next in s_sample:
                    r = -2
                s_sample.append(s)
                
                if done == True:
                    q_target = r
                else:
                    # 下一个状态处的最大动作，a1用greedy_policy，与SARSA不同
                    a1 = self.greedy_policy(self.qvalue, s_next)
                    a1_num = self.find_anum(a1)
                    # qlearning的更新公式TD(0)
                    q_target = r + self.gamma * self.qvalue[s_next, a1_num]
                
                # 利用td方法更新动作值函数
                self.qvalue[s, a_num] = self.qvalue[s, a_num] + alpha * (q_target - self.qvalue[s, a_num])
                
                s = s_next
                a = self.epsilon_greedy_policy(self.qvalue, s, epsilon)
                count += 1
        return self.qvalue

In [ ]:
yuanyang = YuanYangEnv()
TD = TD_RL(yuanyang)

# on-policy方法
qvalue = TD.sarsa(num_iter=5000,alpha=0.1,epsilon=0.8)    # 注意yuanyang中的无效奖励改变为-0.1
# qvalue = TD.qlearning(num_iter=5000,alpha=0.1,epsilon=0.1)
print(qvalue)

# 将行为值函数渲染出来
yuanyang.action_value = qvalue

# 测试学到的策略
flag = 1
s = 0
step_num = 0
path = []

# 将最优路径打印出来
while flag:
    # 渲染路径点
    path.append(s)
    yuanyang.path = path
    a = TD.greedy_policy(qvalue, s)
    print('%d->%s\t' % (s, a), qvalue[s, 0], qvalue[s, 1], qvalue[s, 2], qvalue[s, 3])
    yuanyang.bird_male_position = yuanyang.state_to_position(s)
    yuanyang.render()
    time.sleep(0.25)
    step_num += 1
    s_, r, t = yuanyang.transform(s, a)
    if t == True or step_num > 30:
        flag = 0
    s = s_
    
# 渲染最后的路径点
yuanyang.bird_male_position = yuanyang.state_to_position(s)
path.append(s)
yuanyang.render()
while True:
    yuanyang.render()

sarsa第一次实现最短路径需要的迭代次数为: 232
[[-1.83653111 -1.65476535 -8.58391693 -7.94964431]
 [-1.40380891 -1.38621989 -1.91890742 -5.09515759]
 [-4.10239    -1.54973145 -1.70573084 -3.636559  ]
 [ 0.          0.          0.          0.        ]
 [-0.21166475 -0.1936822  -0.38       -1.        ]
 [-1.         -0.475855   -0.24891075 -1.        ]
 [ 0.          0.          0.          0.        ]
 [-0.01        0.          0.          0.        ]
 [ 1.9         0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [-1.68275503 -1.73883239 -4.5872479  -1.98980704]
 [-1.60819123 -1.39456528 -1.79032219 -1.5671727 ]
 [-3.2951     -1.4083076  -1.62413356 -1.52157753]
 [ 0.          0.          0.          0.        ]
 [-0.246639   -0.2133021  -0.38       -0.209     ]
 [-1.9        -0.21917148 -0.4071     -0.28470941]
 [ 0.          0.          0.          0.        ]
 [-0.01       -0.201805   -0.2        -0.01      ]
 [-0.01       -0.20095    -0.01       -0.01      ]
 [-